<a href="https://colab.research.google.com/github/fvillena/dcc-ia-nlp/blob/master/4-embeddings.sol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word Embeddings

# Actividad 1: Word2vec en tensorflow

Observe el preprocesamiento que se realiza al corpus, la estructura construida con Tensorflow y el procedimiento para entrenar el modelo.


1.   Construya los word embeddings desde los pesos y sesgos del modelo entrenado
2.   Visualice los embeddings bidimensionales en un gráfico de dispersión e interprete los resultados


In [1]:
import tensorflow as tf
import numpy as np

In [2]:
corpus = ['king is a strong man', 
          'queen is a wise woman', 
          'boy is a young man',
          'girl is a young woman',
          'prince is a young king',
          'princess is a young queen',
          'man is strong', 
          'woman is pretty',
          'prince is a boy will be king',
          'princess is a girl will be queen']

In [3]:
def preprocess(text):
  tokens = text.split(" ")
  return [token for token in tokens if len(token) > 2]

In [4]:
def get_vocabulary(corpus):
  vocabulary = []
  for text in corpus:
    vocabulary += text
  return sorted(list(set(vocabulary)))

In [5]:
sentences = list(map(preprocess, corpus))

In [6]:
vocabulary = get_vocabulary(sentences)

In [7]:
WINDOW_SIZE = 2

data = []
for sentence in sentences:
    for idx, word in enumerate(sentence):
        # Acá se toma una ventana de -WINDOWS_SIZE, WINDOWS_SIZE para generar el skip gram. Dado que las 
        # frases son cortas, se utiliza min y max para tener cuidado con los límites de la frase. 
        # Además, el +1 en el límite superior es para considerar el índice de la propia palabra en cuestión 
        # (probar qué ocurre cuando se elimina dicho +1)
        for neighbor in sentence[max(idx - WINDOW_SIZE, 0) : min(idx + WINDOW_SIZE, len(sentence)) + 1]: 
            if neighbor != word:
                data.append((word, neighbor))

In [8]:
features = np.zeros((len(data),len(vocabulary)),dtype=np.float32)
labels = np.zeros((len(data),len(vocabulary)),dtype=np.float32)
for i,(feature,label) in enumerate(data):
  features[i,vocabulary.index(feature)] = 1
  labels[i,vocabulary.index(label)] = 1

In [9]:
class Word2Vec:
  def __init__(self, vocab_size=0, embedding_dim=2, epochs=10000):
    self.vocab_size=vocab_size
    self.embedding_dim=embedding_dim
    self.epochs=epochs
    self.optimizer = tf.optimizers.SGD(learning_rate=0.1)
  def train(self, x_train=None, y_train=None):
    self.W1 = tf.Variable(tf.random.normal([self.vocab_size, self.embedding_dim]))
    self.b1 = tf.Variable(tf.random.normal([self.embedding_dim])) #bias
 
    self.W2 = tf.Variable(tf.random.normal([self.embedding_dim, self.vocab_size]))
    self.b2 = tf.Variable(tf.random.normal([self.vocab_size]))
 
    for _ in range(self.epochs):
      with tf.GradientTape() as t:
        hidden_layer = tf.add(tf.matmul(x_train,self.W1),self.b1) 
        output_layer = tf.nn.softmax(tf.add( tf.matmul(hidden_layer, self.W2), self.b2))
        cross_entropy_loss = tf.reduce_mean(-tf.math.reduce_sum(y_train * tf.math.log(output_layer), axis=[1]))
 
      grads = t.gradient(cross_entropy_loss, [self.W1, self.b1, self.W2, self.b2])
      self.optimizer.apply_gradients(zip(grads,[self.W1, self.b1, self.W2, self.b2]))
      if(_ % 1000 == 0):
        print(cross_entropy_loss)

In [10]:
w2v = Word2Vec(vocab_size=len(vocabulary), epochs=10000)
w2v.train(features, labels)

tf.Tensor(3.7111897, shape=(), dtype=float32)
tf.Tensor(2.143462, shape=(), dtype=float32)
tf.Tensor(1.8884428, shape=(), dtype=float32)
tf.Tensor(1.8343502, shape=(), dtype=float32)
tf.Tensor(1.8103203, shape=(), dtype=float32)
tf.Tensor(1.7978762, shape=(), dtype=float32)
tf.Tensor(1.7909471, shape=(), dtype=float32)
tf.Tensor(1.7865118, shape=(), dtype=float32)
tf.Tensor(1.7833618, shape=(), dtype=float32)
tf.Tensor(1.7809622, shape=(), dtype=float32)


In [11]:
vectors = w2v.W1 + w2v.b1

In [12]:
import plotly.express as px
px.scatter(x=vectors[:,0],y=vectors[:,1],text=vocabulary)

## Embeddings con gensim

In [13]:
import nltk
import re
import gensim
import gensim.downloader
import logging

Descargamos el corpus Brown

In [14]:
nltk.download('brown')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


True

In [15]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [16]:
corpus = nltk.corpus.brown.sents()

Entrenamos un modelo model sobre el corpus Brown, el cual contiene alrededor de 1.000.000 de palabras.

In [17]:
model = gensim.models.word2vec.Word2Vec(sentences = corpus)

Importamos un modelo de word2vec entrenado sobre google news, este modelo fue entrenado sobre un corpus de 3.000.000.000 de palabras

In [18]:
!wget http://sasiba.uchile.cl/index.php/s/rko0uVC2XeAMm8k/download -O GoogleNews-vectors-negative300.gz --no-check-certificate

--2022-11-21 21:55:52--  http://sasiba.uchile.cl/index.php/s/rko0uVC2XeAMm8k/download
Resolving sasiba.uchile.cl (sasiba.uchile.cl)... 200.89.79.35
Connecting to sasiba.uchile.cl (sasiba.uchile.cl)|200.89.79.35|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://sasiba.uchile.cl/index.php/s/rko0uVC2XeAMm8k/download [following]
--2022-11-21 21:55:52--  https://sasiba.uchile.cl/index.php/s/rko0uVC2XeAMm8k/download
Connecting to sasiba.uchile.cl (sasiba.uchile.cl)|200.89.79.35|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/gzip]
Saving to: ‘GoogleNews-vectors-negative300.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  28.0MB/s    in 58s     

2022-11-21 21:56:51 (27.2 MB/s) - ‘GoogleNews-vectors-negative300.gz’ saved [1647046227/1647046227]



In [19]:
news = gensim.models.KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.gz", binary=True, limit=20000)

## Actividad 2: Exploración de los embeddings

Después de calcular los embeddings, podemos asociar un vector de números reales, de dimensiones conocidas a una palabra de nuestro vocabulario.


1.   ¿Cuántas dimensiones tiene cada vector asociado a las palabras en cada uno de los modelos?
2.   ¿Cuántos vectores hay en cada uno de los modelos?
3.   ¿Existen diferencian en el tamaño del vocabulario del modelo `model` y el modelo news?



In [20]:
news.vectors.shape

(20000, 300)

In [21]:
model.wv.vectors.shape

(15173, 100)

## Actividad 3: Similaridad y analogía

Existen un par de cualidades semánticas de las palabras, las cuales pueden ser fácilmente demostradas a través de operaciones vectoriales sobre el espacio generado por el proceso de cálculo de embeddings.

La similaridad es la métrica de cercanía que tienen 2 palabras, esta característica es fácil de representar a través de la similaridad coseno entre 2 vectores.

La analogía es la relación semántica que tienen 2 palabras, por ejemplo, la palabra "rey" y "reina" están relacionadas por el concepto de "género". Estas analogías se pueden operacionalizar en el espacio vectorial de los word embeddings como la resta de los vectores asociados a las palabras.



1.   Verifique cuáles son las palabras más cercanas a palabras que usted seleccione e interprete la correctitud de las palabras retornadas por los modelos `model` y `news`
2.   Verifique cuál de los modelos resuelve mejor la prueba de analogía: "man" es a "woman" como "king" es a "queen". Invente otra analogía y pruebe si el modelo la puede resolver.
3.   Según sus pruebas, ¿el tamaño del corpus de entrenamiento afecta el rendimiento del modelo?



In [22]:
# así obtenemos las palabras más cercanas a las palabra "woman"
model.wv.most_similar("woman")

[('girl', 0.9576287269592285),
 ('boy', 0.9004236459732056),
 ('man', 0.8761813640594482),
 ('child', 0.8725337982177734),
 ('young', 0.8715532422065735),
 ('distaste', 0.8694295287132263),
 ('old', 0.8571096658706665),
 ('fellow', 0.8547695279121399),
 ('artist', 0.854416012763977),
 ('conversation', 0.8543972373008728)]

In [23]:
news.most_similar("woman")

[('man', 0.7664012908935547),
 ('girl', 0.7494640946388245),
 ('teenager', 0.631708562374115),
 ('lady', 0.6288785934448242),
 ('mother', 0.607630729675293),
 ('boy', 0.5975908041000366),
 ('Woman', 0.5770983695983887),
 ('she', 0.5641393661499023),
 ('person', 0.5470173358917236),
 ('victim', 0.545007586479187)]

In [24]:
result = news.vectors[news.vocab["woman"].index] - news.vectors[news.vocab["man"].index] + news.vectors[news.vocab["king"].index]
news.similar_by_vector(result)

[('king', 0.8449392318725586),
 ('queen', 0.7300517559051514),
 ('princess', 0.6156251430511475),
 ('prince', 0.5777117609977722),
 ('ruler', 0.5247419476509094),
 ('throne', 0.5208988785743713),
 ('monarchy', 0.5182886123657227),
 ('royal', 0.5117030143737793),
 ('kingdom', 0.47109082341194153),
 ('palace', 0.4703429341316223)]

In [25]:
result = model.wv.vectors[model.wv.vocab["woman"].index] - model.wv.vectors[model.wv.vocab["man"].index] + model.wv.vectors[model.wv.vocab["king"].index]
model.wv.similar_by_vector(result)

[('per', 0.6180453300476074),
 ('6,000', 0.5896825790405273),
 ('cent', 0.5840734243392944),
 ('centum', 0.5626726150512695),
 ('atmospheres', 0.5524175763130188),
 ('Phase', 0.5445520281791687),
 ('capita', 0.5371654033660889),
 ('Profile', 0.5300782322883606),
 ('Rhode', 0.5234342813491821),
 ('under', 0.515495777130127)]

## Actividad 4: Reducción de dimensionalidad y visualización

Nuestros vectores tienen más de 2 dimensiones, por lo que no es muy fácil interptretar su ubicación espacial. Utiliza un método de reducción de dimensionalidad que conozcas para reducir hacia 2 dimensiones nuestros vectores y visualiza el resultado.


*   ¿Existen agrupaciones aparentes de palabras dentro del espacio? y si es así, ¿las palabras agrupadas, son similares?



In [26]:
import sklearn.decomposition
news_matrix_2d = sklearn.decomposition.PCA(2).fit_transform(news.vectors[:100,:])
import plotly.express as px
px.scatter(x=news_matrix_2d[:,0],y=news_matrix_2d[:,1],text=list(news.vocab)[:100])